# Projet maison n° 4

In [165]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas, pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [197]:
# names us
def df_names_us():
    liste_df = []
    for year in range(1880,2020):
        df = pd.read_csv(f"names/yob{year}.txt", names=["name", "gender", "births"])
        df["year"] = year
        liste_df.append(df)
    df = pd.concat(liste_df).reset_index()
    return(df.reindex(columns=["year","name", "gender", "births"]))

In [198]:
df = df_names_us()
df

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
1989396,2019,Zyheem,M,5
1989397,2019,Zykel,M,5
1989398,2019,Zyking,M,5
1989399,2019,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [231]:
# names fr
def df_names_fr():
    df = pd.read_csv("nat2019_csv/nat2019.csv", sep=";", keep_default_na=False)
    df = df.reindex(columns=["annais", "preusuel", "sexe", "nombre"])
    
    df.columns = ["year","name", "gender", "births"]
    df["births"] = df["births"].astype(int)
    
    df = df[df["year"]!="XXXX"]
    df["year"] = df["year"].astype(int)
    
    df = df[(df["name"]!= "_PRENOMS_RARES") & (df["name"].str.len() > 1)]
    df.name = df.name.apply(lambda s: s.title())
    
    df["gender"] = df["gender"].map({1:"M", 2:"F"})
    
    df = df.sort_values(["year", "gender", "births","name"], ascending=[True, True, False, True])
    df.reset_index(drop=True, inplace=True)
    return(df)

df = df_names_fr()
df

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
615907,2019,Éliel,M,3
615908,2019,Énes,M,3
615909,2019,Énoa,M,3
615910,2019,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [235]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export_20201103.csv", sep=";", index_col=0, decimal=',', skiprows=range(1,6), na_values="-", parse_dates={"Date":[0]})
    df.rename(columns={"Roupie Indienne (100 paise)": "Roupie Indienne (INR)"})
    df.columns = [x[-5:].strip("()") for x in df.columns]
    df = df[df.notnull().any(axis=1)]
    return(df[devises])

In [236]:
df_taux_change(["CHF","GBP", 'USD'])

,CHF,GBP,USD
Date,,,
2020-03-11,1.0709,0.90042,1.1702
2020-02-11,1.0695,0.90053,1.1652
2020-10-30,1.0698,0.90208,1.1698
2020-10-29,1.0684,0.90430,1.1704
2020-10-28,1.0693,0.90662,1.1727
...,...,...,...
1999-08-01,1.6138,0.70940,1.1659
1999-07-01,1.6165,0.70585,1.1632
1999-06-01,1.6116,0.70760,1.1743


### Tests

In [240]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [241]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [242]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.653s

OK
